# Trainingspipeline 3

- Zeiträume: 50 Jahre, auf 2.15 Mio Tokens normalisiert
- ohne Bigramme
- vector_size: 300
- window: 20

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../../Korpus/Korpus/corpus_norm.csv', sep=',', encoding='utf-8')

In [4]:
df.shape

(134168, 11)

In [5]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')
df.cleaned_text = df.cleaned_text.fillna('')

In [6]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [7]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1850']
df4 = df_periods['1851-1900']
df5 = df_periods['1901-1950']
df6 = df_periods['1951-2000']
df7 = df_periods['2001-2021']

## Training von Word2Vec

In [8]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [9]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 20                        # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1
negative = 10                      # negative sampling, how many “noise words” should be drawn

### Zeitraum 1: 1700-1750

In [10]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [11]:
%%time
sentences1 = tokenize_text(text1)     # lemmatisierten Text bereinigen und tokenisieren (Ziel: Liste von tokenisierten Sätzen)

Wall time: 12.4 s


In [12]:
print(sentences1[10])

['mettere', 'tasca', 'senza', 'avvedermene', 'viglietto', 'ora', 'rimare', 'perch', 'restare', 'privo', 's', 'bello', 'cosa']


In [13]:
len(sentences1)

82863

In [14]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 5min 34s


In [15]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7688763737678528),
 ('riempiè', 0.7464150786399841),
 ('macello', 0.730184018611908),
 ('costernazione', 0.7222709655761719),
 ('vandali', 0.7196207046508789),
 ('pestilenza', 0.717168927192688),
 ('sarmati', 0.7162801623344421),
 ('saccheggio', 0.7160423398017883),
 ('trucidare', 0.714810848236084),
 ('milanesi', 0.7110094428062439)]

In [16]:
# trainiertes Modell speichern

w2v1.save(os.path.join('trained_models/Word2Vec5', '5w2v1.model'))

### Zeitraum 2: 1751-1800

In [17]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [18]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 13.6 s


In [19]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 4min 45s


In [20]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('funestare', 0.8481120467185974),
 ('atterrire', 0.8478063344955444),
 ('imminente', 0.8444787859916687),
 ('attentato', 0.8366541266441345),
 ('codardia', 0.8362211585044861),
 ('tregua', 0.8310586810112),
 ('vilt', 0.8301877975463867),
 ('invadere', 0.8253514766693115),
 ('riaccendere', 0.8247053623199463),
 ('sedizione', 0.823040783405304)]

In [21]:
w2v2.save(os.path.join('trained_models/Word2Vec5', '5w2v2.model'))

### Zeitraum 3: 1801-1850

In [22]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [23]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 11.4 s


In [24]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 5min 29s


In [25]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('prorompere', 0.8131869435310364),
 ('forsennato', 0.8046298027038574),
 ('codardi', 0.8017115592956543),
 ('represso', 0.7991253137588501),
 ('pugnar', 0.7985993027687073),
 ('ansia', 0.7982293367385864),
 ('disingannare', 0.7962737679481506),
 ('infernale', 0.7950798869132996),
 ('improvvido', 0.7950248718261719),
 ('irresistibile', 0.7941353917121887)]

In [26]:
w2v3.save(os.path.join('trained_models/Word2Vec5', '5w2v3.model'))

### Zeitraum 4: 1851-1900

In [27]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [28]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 13.2 s


In [29]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 4min 6s


In [30]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('parossismo', 0.780049741268158),
 ('istantaneo', 0.7650575041770935),
 ('dissimul', 0.7573209404945374),
 ('strazio', 0.7554406523704529),
 ('dovevasi', 0.7533871531486511),
 ('cocente', 0.7530575394630432),
 ('invasare', 0.751238226890564),
 ('tentatore', 0.7510941028594971),
 ('raccapriccio', 0.7495812773704529),
 ('attenuare', 0.7449455857276917)]

In [31]:
w2v4.save(os.path.join('trained_models/Word2Vec5', '5w2v4.model'))

### Zeitraum 5: 1901-1950

In [32]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [33]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 13.1 s


In [34]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 4min 21s


In [35]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.8094738125801086),
 ('bestemmia', 0.7934355735778809),
 ('subitaneo', 0.793168842792511),
 ('irrefrenabile', 0.7919957637786865),
 ('spavento', 0.7917354106903076),
 ('straziante', 0.7901254892349243),
 ('raccapriccio', 0.78994220495224),
 ('atterrita', 0.7881133556365967),
 ('ira', 0.7867462038993835),
 ('ingiuria', 0.7832744121551514)]

In [36]:
w2v5.save(os.path.join('trained_models/Word2Vec5', '5w2v5.model'))

### Zeitraum 6: 1951-2000

In [37]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [38]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 19.7 s


In [39]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 4min 52s


In [40]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('feroce', 0.7505583763122559),
 ('galilea', 0.7382466793060303),
 ('serba', 0.7366111874580383),
 ('lager', 0.7365802526473999),
 ('ferocia', 0.7355042695999146),
 ('straziare', 0.735108494758606),
 ('devastatore', 0.7344458103179932),
 ('macabro', 0.7341788411140442),
 ('smisurato', 0.7318776845932007),
 ('squadraccia', 0.7310470342636108)]

In [41]:
w2v6.save(os.path.join('trained_models/Word2Vec5', '5w2v6.model'))

### Zeitraum 7: 2001-2021

In [42]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [43]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 9.57 s


In [44]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 7min 21s


In [45]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('islamista', 0.783943772315979),
 ('carceriere', 0.7809414863586426),
 ('crudele', 0.7794572710990906),
 ('orrore', 0.7742250561714172),
 ('dèi', 0.7739911079406738),
 ('atrocità', 0.7624913454055786),
 ('zii', 0.7606104016304016),
 ('ves', 0.7590230107307434),
 ('trii', 0.7572970986366272),
 ('prigioniero', 0.7554473876953125)]

In [46]:
w2v7.save(os.path.join('trained_models/Word2Vec5', '5w2v7.model'))